In [1]:
####################
#set up packages 
####################
import ee
import geemap
import numpy as np
import ipyleaflet
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import geemap.colormaps as cm
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import Output
from ipyleaflet import WidgetControl




#Nightlight in the Caribbean
### Compairing the month before and after the storm, examinging medium and high density urban settlements 
### Infering the cost of a storm using line of best fit from all recorded nightlight data 


In [2]:

####################################
#country list to loop through
####################################
Map=geemap.Map() #setting up the map to display 
countries = ee.FeatureCollection('FAO/GAUL/2015/level0') #countries is the dataset
Nightlight= ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG").select('avg_rad') #the dataset and band selected
JamaicanNight= Nightlight #the dataset is cutdown to the relivant dates and region removed .filterbounds(roi)
    
countrylist=[
'Antigua and Barbuda',
'Barbados',
'Belize',
'Aruba',
'Dominican Republic', 
'Haiti',
'Puerto Rico',
'Cayman Islands', 
'Dominica',
'Jamaica',
'Trinidad and Tobago', 
'Saint Vincent and the Grenadines',
'Saint Kitts and Nevis',
'Saint Lucia']
storms_df=pd.read_csv('storms_data.csv')
stormlist=storms_df['Event Name'].unique()

interpolationdata=pd.read_csv('interpolationlines.csv')
band = 'avg_rad'






In [4]:
#visualisation parameteres are set up
Palette= cm.palettes.brg

Vis_params= {
    'min': -20,
    'max': 20,
    'palette': ['blue','red','green']}


def threshold(img, level):
    mask=img.lt(level)
    #mask=mask.selfMask()
    return mask

# Start_date='2017-08-01'
# End_date='2017-10-01'

#seting up nightlight VIIRS monthly

Map.add_colorbar(Vis_params, label='percentage difference in nightlight',layer_name='color bar')




In [5]:
t_level=5


def extract_nightlight(image):
    night=image.reduceRegion(reducer=ee.Reducer.sum(), scale=1000).values()
    return night


def func(Country,Storm):
    
    stormname=Storm
    countryname=Country

 
    roi = countries.filter(ee.Filter.eq('ADM0_NAME', countryname)) #roi is a subset of the countries dataset 
    roigeom=roi.geometry() #set up as a geometry to allow .filterBounds and .clip would work in other ways but this is most efficient
    roicenter=roigeom.centroid() #this allows the map to be centered on the center of the roi
    # Map.addLayer(roigeom,{},'Region of Interest')
    Map.centerObject(roi,5)


    startyear=storms_df.loc[storms_df['Event Name']==stormname]['Start Year'].values[0]
    startmonth=storms_df.loc[storms_df['Event Name']==stormname]['Start Month'].values[0]
    
    if startmonth < 10:
        Start_date="%i-0%i-01"%(startyear,startmonth)
    if startmonth >=10:
         Start_date="%i-%i-01"%(startyear,startmonth)

        
    if (startmonth+1) < 10:
        End_date="%i-0%i-01"%(startyear,startmonth+1)
    if (startmonth+1) >=10:
        End_date="%i-%i-01"%(startyear,startmonth+1)
    if (startmonth+1) >12:
        End_date="%i-%i-01"%(startyear+1,startmonth+1-12)



    # print(type(roi))
    #before image is set up
    Before = ee.Image(JamaicanNight.filterDate(Start_date).mean())
    #Map.addLayer(Before.clipToCollection(roi),{},'Before2')
    Before = Before.clipToCollection(roi)

    #before image is thresholded and mask set up
    Before_mask = threshold(Before, t_level)
    Before_mask = Before_mask.clipToCollection(roi)
    Before_masked=Before.where(Before_mask,0)

    #After image set up
    After = ee.Image(JamaicanNight.filterDate(End_date).mean())
    #Map.addLayer(After.clipToCollection(roi),{},'After 2')
    After = After.clipToCollection(roi)

    #After image is thresholded and mask set up 
    # After_mask = threshold(After, t_level)
    # After_mask = After_mask.clip(roi)
    After_masked=After.where(Before_mask,0) #here before mask is used because if the light drops to less than threshold in interesting areas this is useful information

    #Before and after added to map
    # Map.addLayer(Before_mask,{},'Before mask')
    # Map.addLayer(Before_masked,{},'Before')
    # Map.addLayer(After_masked,{},'After')


    #the calculation for the percentage difference set up (could be written as 1 equation)
    pd=After_masked.subtract(Before_masked)
    pd=pd.divide(Before_masked)
    pd=pd.multiply(100)

    #pd mask is set up, the .mask function every pixel with or lower value in the given image are transparent, which is why before.not() is used
    pd_mask = Before_mask.Not()
    pd_mask = pd_mask.clipToCollection(roi)
    pd_masked=pd.mask(pd_mask)

    #visualisation for export set up
    export=pd_masked.visualize(**{
        'min': -20,
        'max': 20,
        'palette': ['blue','red','green']}).clip(roi) #was manually entered for some reason the previous parameters where not acepted because of the palette

    #added to map with color bar
    Map.addLayer(pd_masked,Vis_params,'Difference')

    change_in_nightlight = After_masked.subtract(Before_masked).reduceRegion(reducer=ee.Reducer.sum(), geometry=roigeom, scale=1000).values().getInfo()
    cost=float(interpolationdata.loc[interpolationdata['Country']==countryname]['constant'].tolist()[0])+float(interpolationdata.loc[interpolationdata['Country']==countryname]['multiplier'].tolist()[0])*change_in_nightlight[0]
    print("Estimated change in GDP following storm in billions of US dollars $",cost)

    # # plotting nightlight over time 
    # nightlights=[]
    # dates=[]
    
    # country_of_intrest=JamaicanNight.filterBounds(roigeom).filterDate("01/01/2015-04/01/2015")
    # nightlights=JamaicanNight.map(lambda x: 
    #                                          x.clip(roi)\
    #                                           .reduceRegion(geometry=roigeom,reducer=ee.Reducer.mean(),scale=1000)\
    #                                           .get(band)).getInfo()
   
    # print(nightvalue)
                
    
    # print(nightlights.getInfo())
    # dates = JamaicanNight.map(lambda image:
    #                        ee.Feature(None, {'date': image.date().format('YYYY-MM-dd')}))\
    #                                                  .distinct('date').aggregate_array('date').getInfo()

    # print(dates)
    # print(nightlights,dates)


    # t = np.arange(0.0, 2.0, 0.01)
    # s = 1 + np.sin(2 * np.pi * t)

    # fig, ax = plt.subplots()
    # ax.plot(t, s)

    # ax.set(
    #     xlabel='time (s)', ylabel='voltage (mV)', title='About as simple as it gets, folks'
    # )
    # ax.grid()

    # # Create an output widget to host the plot
    # output_widget = Output()

    # # Show the plot on the widget
    # with output_widget:
    #     output_widget.clear_output()
    #     plt.show()

    # # Add the widget as a control to the map
    # output_control = WidgetControl(widget=output_widget, position="bottomright")
    # Map.add_control(output_control)
    #     # Map.addLayer(export,{},'export')



    #estimated cost

    
        
    

dropdown = interact(func, Country=countrylist, Storm=stormlist)
Map


interactive(children=(Dropdown(description='Country', options=('Antigua and Barbuda', 'Barbados', 'Belize', 'A…

Map(center=[17.29488411286719, -61.80079216571587], controls=(WidgetControl(options=['position', 'transparent_…

In [6]:
# countryname="Saint Lucia"
# #cost=float(interpolationdata.loc[interpolationdata['Country']==countryname]['constant'].tolist()[0])
# #+float(interpolationdata.loc[interpolationdata['Country']==countryname]['multiplier'].tolist()[0])*change_in_nightlight[0]
# interpolationdata.loc[interpolationdata['Country']==countryname]

,Country,constant,multiplier
13,Saint Lucia,40.25205,111.1567
